In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

from bs4 import BeautifulSoup

from datetime import datetime
import pandas as pd

from tqdm import trange, tqdm_notebook

In [ ]:
#########################################################################################
# Copied/pasted from https://realpython.com/python-web-scraping-practical-introduction  #
#########################################################################################

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [ ]:
# Extract the total number of pages to be scraped
base_url = 'https://www.biorxiv.org/content/early/recent'
raw_html = simple_get(base_url)
html = BeautifulSoup(raw_html, 'html.parser')

page_count = int(html.select('li.pager-last > a')[0].text)
print(page_count)

In [ ]:
# Scrape all the pages (takes about 2 seconds per page)
preprints = []

for i in trange(page_count):
    url = 'https://www.biorxiv.org/content/early/recent?page={}'.format(i)
    
    raw_html = simple_get(url)
    html = BeautifulSoup(raw_html, 'html.parser')

    for div in html.find_all(name='div', attrs={'class': 'highwire-list-wrapper'}):
        pub_date_txt = div.find('h3').text
        # February 5, 2019 -> 20190205
        pub_date = datetime.strptime(pub_date_txt, '%B %d, %Y')

        for li in div.select('div > ul > li'):
            p_div = li.find('div', attrs={'class': 'highwire-article-citation'})
            p_attrs = p_div.attrs
            
            # some articles don't have type annotation
            p_type = p_div.select('div.highwire-cite-metadata > span.highwire-cite-metadata-overline')
            if len(p_type) < 1:
                p_type_str = ''
            else:
                p_type_str = p_type[0].text
           
            # really old articles have missing titles
            p_title = p_attrs.get('title', None)
            if p_title is None:
                p_title = p_attrs.get('oldtitle', None)

            preprint = {'node_id': p_attrs['data-node-nid'],
                        'version_id': p_attrs['data-pisa'].split(';')[1],
                        'master_id': p_attrs['data-pisa-master'].split(';')[1],
                        'title': p_title,
                        'preprint_type': p_type_str,
                        'is_revision': not p_attrs['data-pisa'].endswith('v1'),
                        'pub_date': pub_date_txt,
                        'pub_date_year': pub_date.year,
                        'pub_date_month': pub_date.month,
                        'pub_date_day': pub_date.day}

            preprints.append(preprint)
        
preprints_df = pd.DataFrame.from_dict(preprints)
preprints_df.to_csv('preprints.csv')

In [ ]:
print('Total preprints scraped: {}'.format(len(preprints)))